In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import tensorflow as tf
import os


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%%time

import datasets #huggingface datasets

print("Loading Dataset")
data = datasets.load_dataset("raygx/Nepali-Extended-Text-Corpus")
print(data)

Loading Dataset


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/raygx--Nepali-Extended-Text-Corpus-3fb987a1cedf9271/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'id'],
        num_rows: 14613470
    })
    test: Dataset({
        features: ['text', 'id'],
        num_rows: 14629
    })
})
CPU times: user 1min 10s, sys: 19.5 s, total: 1min 30s
Wall time: 1min 34s


In [ ]:
##### BAG of words computation
from tqdm.auto import tqdm

bag_of_words = set()

for i in tqdm(range(0,data['train'].num_rows,50000)):
    j = i+100000
    j = j if j<data['train'].num_rows else data['train'].num_rows

    bag_of_words = set(list(bag_of_words)+(" ".join(data['train'].select(range(i,j))['text'])).split())


len(bag_of_words)  ## 4966875 words in the bag

## Preparing BERT training data

In [3]:
from transformers import AutoTokenizer

context_length = 512

print("Loading Tokenizer")

tokenizer = AutoTokenizer.from_pretrained('raygx/BERT_Nepali_Tokenizer')
tokenizer.add_special_tokens({'pad_token': '[PAD]','unk_token':'[UNK]'})
tokenizer.model_max_length = context_length

tokenizer

Loading Tokenizer


PreTrainedTokenizerFast(name_or_path='raygx/BERT_Nepali_Tokenizer', vocab_size=50000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [4]:
%%time

def preprocess_function(rows):
    concatenated_rows = tokenizer(tokenizer.sep_token.join(rows['text']))

    total_length = len(concatenated_rows[list(concatenated_rows.keys())[0]])
    splits = int(total_length/context_length)    
    
    result = {
        k: np.array_split(t[:splits*context_length],splits)
        for k, t in concatenated_rows.items()
    }
    
    if total_length > splits*context_length:
        for k, t in concatenated_rows.items():
            result[k].append(concatenated_rows[k][-(context_length):])
    
    result["labels"] = result["input_ids"].copy()  # unnecessary space consumption can be done before training
    return result

num_proc = os.cpu_count()

print("Tokenizing the data")
lm_data = data.map(
        preprocess_function,
        batched=True,
        num_proc=num_proc,
        remove_columns=data["train"].column_names,
        batch_size=5000
    )

print(lm_data)
lm_data.save_to_disk('BERT_Training_Data')

Tokenizing the data
     

#0:   0%|          | 0/731 [00:00<?, ?ba/s]

#1:   0%|          | 0/731 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (149913 > 512). Running this sequence through the model will result in indexing errors


#2:   0%|          | 0/731 [00:00<?, ?ba/s]

...

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (112459 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (115283 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (110572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (117139 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 860233
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 891
    })
})
CPU times: user 17.9 s, sys: 10.1 s, total: 28 s
Wall time: 28min 50s
